In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import faiss
import streamlit as st
from langchain import PromptTemplate
from langchain_community.llms import CTransformers

In [6]:
doc_loader = DirectoryLoader(path="data/", glob="*.pdf", loader_cls=PyPDFLoader)
documents = doc_loader.load()

In [2]:
import os 
os.chdir("../")

In [8]:
# split the documents into chunks 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})

C:\Users\eigen\AppData\Local\Temp\ipykernel_10164\1360172283.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
c:\Users\eigen\anaconda3\envs\tf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("vector_store/faiss_db")

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\eigen\AppData\Local\Temp\ipykernel_10164\3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [12]:
llm = CTransformers(
    model="models/llama-2-7b-chat.ggmlv3.q4_0.bin", 
    model_type="llama", 
    config={"max_new_tokens":28, "temperature":0.01}
)

In [ ]:
chain = ConversationalRetrievalChain(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store(search_kwargs={"k":3}), 
    memory=memory
)

In [ ]:
# chat function 
def conversation_chat(query): 
    result = chain({"question":query, "chat_history":st.session["history"]})
    st.session.state["history"].append((query, result["answer"]))
    return result["answer"]



In [30]:
import math 

math.isqrt(10)

3

In [31]:
int(9.834)

9

In [3]:
from src.model import final_result

In [4]:
final_result("What is acne?")

c:\Users\eigen\Desktop\LlaMA-MedBot\src\model.py:58: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
c:\Users\eigen\anaconda3\envs\tf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\eigen\Desktop\LlaMA-MedBot\src\model.py:48: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide 

ValidationError: 4 validation errors for RetrievalQA
combine_documents_chain
  Field required [type=missing, input_value={'llm': CTransformers(cli... and nothing more.\n")}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
llm
  Extra inputs are not permitted [type=extra_forbidden, input_value=CTransformers(client=<ctr...8, 'temperature': 0.01}), input_type=CTransformers]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
chain_type
  Extra inputs are not permitted [type=extra_forbidden, input_value='stuff', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
chain_type_kwargs
  Extra inputs are not permitted [type=extra_forbidden, input_value={'prompt': PromptTemplate...r and nothing more.\n")}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden